In [1]:
import socket
import threading
import sys
import pickle
import os

In [ ]:
class Servidor():
    
    def __init__(self, host=socket.gethostname(), port=59989):
        self.clientes = []
        self.sock = socket.socket()
        self.sock.bind((str(host), int(port)))
        self.sock.listen(20)
        self.sock.setblocking(False)
        self.usuarios = []

        aceptar = threading.Thread(target=self.aceptarC)
        procesar = threading.Thread(target=self.procesarC)

        aceptar.daemon = True
        aceptar.start()

        procesar.daemon = True
        procesar.start()

        while True:
            msg = input('SALIR = Q\n')
            if msg == 'Q':
                print("**** TALOGOOO *****")
                self.sock.close()
                sys.exit()
            else:
                pass

    def broadcast(self, msg, cliente):
        for c in self.clientes:
            try:
                if c != cliente:
                    c.send(msg )
            except:
                self.clientes.remove(c)

    def aceptarC(self):
        while True:
            try:
                conn, addr = self.sock.accept()
                print(f"\nConexion aceptada via {conn}\n")
                conn.setblocking(False)
                self.clientes.append(conn)
            except:
                pass

    def procesarC(self):
        print("Procesamiento de mensajes iniciado")
        while True:
            if len(self.clientes) > 0:
                for c in self.clientes:
                    try:
                        data = c.recv(32)
                        print('data',pickle.loads(data))
                        msg = pickle.loads(data);
                        if data:
                            if 'nickname:' in msg:
                                self.usuarios.append(msg.replace('nickname:',''))
                                print("Usuarios conectados:",self.usuarios)
                            else:
                                self.broadcast(data,c)
                    except:
                        pass

s = Servidor()